In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdummysubjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.ch

In [5]:
from tqdm import tqdm
from edc_constants.constants import NO
from edc_base.utils import age
from flourish_caregiver.models import CaregiverChildConsent
from flourish_child.models import InfantHIVTesting, InfantHIVTesting9Months, \
    InfantHIVTestingAge6To8Weeks, InfantHIVTestingOther

infant_hiv_test_objs = InfantHIVTesting.objects.all()

for infant_hiv_test_obj in tqdm(infant_hiv_test_objs):
    if (infant_hiv_test_obj.child_tested_for_hiv == NO or
            infant_hiv_test_obj.child_tested_for_hiv == 'Dont_know'):
        continue
    subject_identifier = infant_hiv_test_obj.child_visit.subject_identifier
    child_consent = CaregiverChildConsent.objects.filter(
        subject_identifier=subject_identifier).latest('consent_datetime')
    if not (child_consent.child_dob and infant_hiv_test_obj.child_test_date):
        continue
    age_at_test = age(child_consent.child_dob, infant_hiv_test_obj.child_test_date)

    options = {
        'child_visit': infant_hiv_test_obj.child_visit,
        'report_datetime': infant_hiv_test_obj.report_datetime,
        'child_tested_for_hiv': infant_hiv_test_obj.child_test_date,
        'child_test_date_estimated': infant_hiv_test_obj.child_test_date_estimated,
        'results_received': infant_hiv_test_obj.results_received,
        'recall_result_date': infant_hiv_test_obj.recall_result_date,
        'received_date': infant_hiv_test_obj.received_date,
        'result_date_estimated': infant_hiv_test_obj.result_date_estimated,
        'hiv_test_result': infant_hiv_test_obj.hiv_test_result,
        'additional_comments': infant_hiv_test_obj.additional_comments,
    }
    #test done at 6-8 weeks
    if age_at_test.years == 0 and age_at_test.months == 2:
        test_obj = InfantHIVTestingAge6To8Weeks.objects.create(
            **options
        )
    #test done at 9 months
    elif age_at_test.years == 0 and age_at_test.months == 9:
        test_obj = InfantHIVTesting9Months.objects.create(
            **options
        )
    #test done at 18 months
    elif age_at_test.years == 1 and age_at_test.months == 6:
        test_obj = InfantHIVTesting9Months.objects.create(
            **options
        )
    #test done at birth
    elif age_at_test.years == 0 and age_at_test.months == 0:
        test_obj = InfantHIVTesting9Months.objects.create(
            **options
        )
    else:
        test_obj = InfantHIVTestingOther.objects.create(
            **options
        )
    test_obj.save()


100%|██████████| 226/226 [00:02<00:00, 82.42it/s] 
